# Speed up your NetworkX code with GraphBLAS

NetworkX is a pure python library for graph theory and network science by design. The base data structure for graphs in NetworkX are python dictionaries. This comes with a lot of advantages like extremely flexible and abstract ways of modelling your graph data, but one of the shortcomings is the performance. Native python objects and looping over them can slow down some things. NetworkX does use the sparse.array module in SciPy wherever it can to speed up algorithms but this still requires moving data from NetworkX to SciPy which can be an expensive operation.

With NetworkX 3.0 we have started experimenting with dispatching certain algorithms to GraphBLAS. GraphBLAS is a specification to do graph operations using tools in linear algebra. We are still in early stages of designing the API and dispatch mechanism to multiple different backends. This dispatch mechanism will also let users run code on different hardwares like GPUs. We hope to soon have a NetworkX backend which can link into CuGraph.

In [ ]:
import networkx as nx

In [ ]:
G = nx.erdos_renyi_graph(1000, 0.5)

In [ ]:
nx.betweenness_centrality(G)

In [ ]:
type(G)

In [ ]:
import graphblas_algorithms as gb

In [ ]:
# Convert to a GraphBLAS Graph object from NetworkX
GB = gb.Graph.from_networkx(G)

In [ ]:
type(GB)

In [ ]:
# A backend Graph doesn't need to be a subclass of nx.Graph
issubclass(gb.Graph, nx.Graph)

With this setup, backend implementors do not need to worry about the data structures inside NetworkX. They just need to provide an interface to the NetworkX API and have a class attribute `__networkx_plugin__` which will help NetworkX dispatch the algorithm to the correct backend.

In [ ]:
GB.__networkx_plugin__

## Let's trying some dispatching examples

In [ ]:
%%time
_ = nx.pagerank(G)

In [ ]:
%%time
_ = nx.pagerank(GB)

In [ ]:
%%time
_ = nx.square_clustering(G)

In [ ]:
%%time
_ = nx.square_clustering(GB)

In [ ]:
%%time
_ = nx.betweenness_centrality(G)

In [ ]:
%%time
_ = nx.betweenness_centrality(GB)

It will be hard to cover all the algorithms in NetworkX by a backend as there is a long tail of algorithms implemented in NetworkX.

## Use Dispatching in your codebases

Just convert everything to a graphblas graph for now (where ever you have create a networkx object change it to a graphblas object). `graphblas-algorithms` does cover a good number of algorithms so it's possible that everything just works (tm) with just a single line change in your codebase.

# Let's write our own backend: `MetaGraphThatWrapsEveryOtherGraphObject`

So we know that there are some algorithms which are not implemented by the `graphblas` backend, but can we design a backend which will always route us to the fastest implementation and not raise a `NetworkXNotImplemented`?

- One way to create this `MetaGraphThatWrapsEveryOtherGraphObject` would be to create a new backend that holds pointers to the networkx graph object and all the other backend implementations objects (the object with the `__networkx_plugin__` class variable).

- Currently we have only two backends (NetworkX `Dict` and `graphblas`), how would you design the priority heuristics in the case of multiple backends (CPU, GPU, single node CPU, multi node CPU/GPU .....)?

- This is an interesting excercise as we (NetworkX and backend plugin developers) are still experimenting and figuring out how to implement the next part of NetworkX dispatching. If you have ideas we would love to talk more about them! (We have weekly sync up meetings <link to calendar>)

For this tutorial let's focus on making this work with just NetworkX `Dict` and `graphblas` objects. We can assume that a `graphblas` implementation (if it exists) will always be faster than the native python `Dict`.

The following code snippet should work with our new backend:
``` python
G_nx = nx.erdos_renyi_graph(1000, 0.5)
G_gb = gb.Graph.from_networkx(G_nx)
G = MetaWrap(G_nx, G_gb)
nx.pagerank(G)
nx.betweenness_centrality(G)
```

In [ ]:
from metawrap.graph import MetaWrap

In [ ]:
G_nx = nx.erdos_renyi_graph(1000, 0.5)
G_gb = gb.Graph.from_networkx(G_nx)

In [ ]:
G = MetaWrap(G_nx, G_gb)

In [ ]:
_ = nx.pagerank(G)

In [ ]:
_ = nx.betweenness_centrality(G)